In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pickle


In [ ]:
from google.colab import drive
drive.mount('/content/Drive')

Mounted at /content/Drive


Dans ce détecteur automatique, nous essaierons d'automatiser le processus de classification.
après les expériences précédentes, nous avons vu que la précision des modèles de perception sur texte dépasse le modèle général lorsque l'humidité est comprise entre 0 et 60 ([0,60]) mais la précision générale du modèle l'emporte lorsque l'humidité est comprise entre 61 et 100 ([61,100]) .
nous allons donc créer une combinaison des trois premiers modèles d'approche de la conscience du contexte et du modèle général et classer par l'un des modèles en fonction de l'humidité.
les étapes à suivre sont donc les suivantes:
1. importer les quatre modèles et le jeu de données + préparer les données à intégrer dans nos modèles.
2. choisissez au hasard une ligne à partir de celui-ci (au lieu de saisir les données manuellement)
3. retirez la colonne de pluie demain et extrayez l'humidité de la rangée.
4. classer le point de données (ligne) dans l'un des modèles en fonction de l'humidité (savoir s'il va pleuvoir demain ou non).
5. imprimez s'il va pleuvoir demain ou non.
******
la séparation sera la suivante:
1. Humidité [0,20]: modèle de prise de conscience du contexte 1.
2. Humidité [21,40]: modèle de prise de conscience du contexte 2.
3. Humidité [41,60]: modèle de prise de conscience du contexte 3.
4. Humidité [61,100]: modèle général.

# 1. importer les quatre modèles et le jeu de données + préparer les données à intégrer dans nos modèles.

In [ ]:
CA_model1=pickle.load(open('/content/Drive/MyDrive/context awareness models/weather models/RF/model 1 context(Humidity) optimized acc_97_073.sav', 'rb'))
CA_model2=pickle.load(open('/content/Drive/MyDrive/context awareness models/weather models/RF/model 2 context(Humidity) optimized acc_93_298.sav', 'rb'))
CA_model3=pickle.load(open('/content/Drive/MyDrive/context awareness models/weather models/RF/model 3 context(Humidity) optimized acc_87_119.sav', 'rb'))
G_model=pickle.load(open('/content/Drive/MyDrive/weather models/model_optimized_RandomForest_weather_84-494.sav', 'rb'))

In [ ]:
df = pd.read_csv("/content/Drive/MyDrive/context awareness datasets/weather dataset/weatherAUS.csv")


In [ ]:
df.RainToday[df.RainToday == 'Yes'] = 1
df.RainToday[df.RainToday == 'No'] = 0

df.RainTomorrow[df.RainTomorrow == 'Yes'] = 1
df.RainTomorrow[df.RainTomorrow == 'No'] = 0

In [ ]:
# remplir les valeurs manquantes pour les variables numériques avec la moyenne
df["MinTemp"]= df["MinTemp"].fillna(df["MinTemp"].mean())
df["MaxTemp"]= df["MaxTemp"].fillna(df["MaxTemp"].mean())
df["Evaporation"]= df["Evaporation"].fillna(df["Evaporation"].mean())
df["Sunshine"]= df["Sunshine"].fillna(df["Sunshine"].mean())
df["WindGustSpeed"]= df["WindGustSpeed"].fillna(df["WindGustSpeed"].mean())
df["Rainfall"]= df["Rainfall"].fillna(df["Rainfall"].mean())
df["WindSpeed9am"]= df["WindSpeed9am"].fillna(df["WindSpeed9am"].mean())
df["WindSpeed3pm"]= df["WindSpeed3pm"].fillna(df["WindSpeed3pm"].mean())
df["Humidity9am"]= df["Humidity9am"].fillna(df["Humidity9am"].mean())
df["Humidity3pm"]= df["Humidity3pm"].fillna(df["Humidity3pm"].mean())
df["Pressure9am"]= df["Pressure9am"].fillna(df["Pressure9am"].mean())
df["Pressure3pm"]= df["Pressure3pm"].fillna(df["Pressure3pm"].mean())
df["Cloud9am"]= df["Cloud9am"].fillna(df["Cloud9am"].mean())
df["Cloud3pm"]= df["Cloud3pm"].fillna(df["Cloud3pm"].mean())
df["Temp9am"]= df["Temp9am"].fillna(df["Temp9am"].mean())
df["Temp3pm"]= df["Temp3pm"].fillna(df["Temp3pm"].mean())

In [ ]:
#Remplir les valeurs manquantes pour les variables catégorielles avec mode
df['RainToday']=df['RainToday'].fillna(df['RainToday'].mode()[0])
df['RainTomorrow']=df['RainTomorrow'].fillna(df['RainTomorrow'].mode()[0])
df['WindDir9am'] = df['WindDir9am'].fillna(df['WindDir9am'].mode()[0])
df['WindGustDir'] = df['WindGustDir'].fillna(df['WindGustDir'].mode()[0])
df['WindDir3pm'] = df['WindDir3pm'].fillna(df['WindDir3pm'].mode()[0])

In [ ]:
# # Suppression de colonnes hautement corrélées
df=df.drop(['Temp3pm','Temp9am','Humidity9am','MaxTemp','Pressure3pm'],axis=1)


In [ ]:
##transformer colonnes catégorielles en colonnes numériques à l'aide de l'encodeur d'étiquette
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df["Location"] = label_encoder.fit_transform(df["Location"])
df["WindDir9am"]= label_encoder.fit_transform(df["WindDir9am"])
df["WindDir3pm"]= label_encoder.fit_transform(df["WindDir3pm"])
df["WindGustDir"] = label_encoder.fit_transform(df["WindGustDir"])

In [ ]:
df['Date']= pd.to_datetime(df['Date'])

In [ ]:
def to_season(m):
    if ((m==12)|(m==1)|(m==2)):
        return 1#"winter"
    elif ((m==3)|(m==4)|(m==5)):
        return 2#"spring"
    elif ((m==6)|(m==7)|(m==8)):
        return 3#"summer"
    elif ((m==9)|(m==10)|(m==11)):
        return 4#"autumn"

df["Date"] = df["Date"].dt.month.map(to_season)



In [ ]:
#maintenant ,nous allons renommer la colonne (Date => Season)
df.rename(columns = {'Date': 'Season'}, inplace = True)


In [ ]:
df.head()

,Season,Location,MinTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity3pm,Pressure9am,Cloud9am,Cloud3pm,RainToday,RainTomorrow
0,1,2,13.4,0.6,5.468232,7.611178,13,44.0,13,14,20.0,24.0,22.0,1007.7,8.000000,4.50993,0,0
1,1,2,7.4,0.0,5.468232,7.611178,14,44.0,6,15,4.0,22.0,25.0,1010.6,4.447461,4.50993,0,0
2,1,2,12.9,0.0,5.468232,7.611178,15,46.0,13,15,19.0,26.0,30.0,1007.6,4.447461,2.00000,0,0
3,1,2,9.2,0.0,5.468232,7.611178,4,24.0,9,0,11.0,9.0,16.0,1017.6,4.447461,4.50993,0,0
4,1,2,17.5,1.0,5.468232,7.611178,13,41.0,1,7,7.0,20.0,33.0,1010.8,7.000000,8.00000,0,0


#2.choisissez au hasard une ligne à partir de celui-ci (au lieu de saisir les données manuellement)    

In [ ]:
data_point = df.sample()
data_point

,Season,Location,MinTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity3pm,Pressure9am,Cloud9am,Cloud3pm,RainToday,RainTomorrow
82590,3,12,5.5,3.8,1.8,4.1,3,48.0,3,14,22.0,15.0,95.0,1008.1,4.447461,4.50993,1,1


#3. retirez la colonne de pluie demain et extrayez l'humidité de la rangée.

In [ ]:
X=data_point.drop("RainTomorrow", axis = 1)
humidity=X.iloc[0]['Humidity3pm']
X

,Season,Location,MinTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity3pm,Pressure9am,Cloud9am,Cloud3pm,RainToday
82590,3,12,5.5,3.8,1.8,4.1,3,48.0,3,14,22.0,15.0,95.0,1008.1,4.447461,4.50993,1


#4. classer le point de données (ligne) dans l'un des modèles en fonction de l'humidité (savoir s'il va pleuvoir demain ou pas).

In [ ]:
if (humidity <=20):
  prediction=CA_model1.predict(X)
elif ((humidity >20 ) & (humidity <=40)):
  prediction=CA_model2.predict(X)
elif ((humidity >40 ) & (humidity <=60)):
  prediction=CA_model3.predict(X)
elif (humidity >60):
  prediction=G_model.predict(X)


#5. Pr.édire s'il va pleuvoir demain ou pas !!!

In [ ]:
if (prediction[0]==1) : 
  print("It's going to rain tomorrow ☔")
else :
  print ("It's NOT going to rain tomorrow ☀")

It's going to rain tomorrow ☔
